In [36]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry

from api_key import APIKEY

API_KEY = APIKEY

In [37]:
CO = ['mp-3414', 'mp-542001', 'mp-28845', 'mp-561423', 'mp-14276', 'mp-29590', 'mp-556442', 'mp-27980']
CH4 = ['mp-680722', 'mp-31107', 'mp-1095378', 'mp-23569', 'mp-30003', 'mp-1190864', 'mp-23084', 'mp-15511', 'mp-7128', 'mp-28855', 'mp-561561', 'mp-1079364', 'mp-542115', 'mp-31232', 'mp-755116', 'mp-1091363', 'mp-558910', 'mp-14037', 'mp-29698', 'mp-1095283', 'mp-20805', 'mp-1113861', 'mp-16834', 'mp-778924', 'mp-556120', 'mp-20727', 'mp-14368', 'mp-1819', 'mp-22467', 'mp-753747', 'mp-15750', 'mp-554435', 'mp-1105700', 'mp-1217378', 'mp-680219', 'mp-17637', 'mp-759342', 'mp-23446', 'mp-779156', 'mp-17677', 'mp-685353', 'mp-230', 'mp-31069', 'mp-1111368', 'mp-759307', 'mp-624234', 'mp-10798', 'mp-760188', 'mp-7094']
CHOOH = ['mp-20015', 'mp-14367', 'mp-541114', 'mp-28299', 'mp-562338', 'mp-554944', 'mp-768936', 'mp-29193']


In [38]:
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in CH4:
        material_entries = mpr.materials.search(material_ids=[mp_id])

        if material_entries:
            mp_entries.append(material_entries[0])

chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')]
materials_gga = [material + "-GGA" for material in CH4]
materials_id = CH4


Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

...

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [39]:
from pymatgen.core.composition import Composition

def remove_o_and_h(composition):
    new_composition_elements = []
    for element, count in composition.get_el_amt_dict().items():
        if element not in ['O', 'H']:
            new_composition_elements.append(f"{element}{int(count)}")
    new_composition_str = ' '.join(new_composition_elements)
    return Composition(new_composition_str)

filtered_molecules = [remove_o_and_h(molecule) for molecule in composition_list]

In [40]:
from pymatgen.core.composition import Composition

def calculate_percent_composition(molecule):
    total_atoms = sum(molecule.values())
    return {element.symbol: count / total_atoms for element, count in molecule.items()}

percent_compositions = [calculate_percent_composition(molecule) for molecule in filtered_molecules]

In [41]:
pbx_data = []
pbx_entries = []

total_iterations = len(chemsys_list)  

with MPRester(API_KEY) as mpr:
    for index, (i, x) in enumerate(zip(chemsys_list, percent_compositions)):
        try:
            pourbaix_entries = mpr.get_pourbaix_entries(i)
            pbx_entries.append(pourbaix_entries)
            pbx_diagram = PourbaixDiagram(entries=pourbaix_entries, comp_dict=x)
            pbx_data.append(pbx_diagram)
        except Exception as e:
            pbx_data.append('error')
        
        percent_complete = ((index + 1) / total_iterations) * 100
        print(f'Percent complete: {percent_complete:.2f}%')

Retrieving ThermoDoc documents:   0%|          | 0/349 [00:00<?, ?it/s]

Percent complete: 2.04%


Retrieving ThermoDoc documents:   0%|          | 0/134 [00:00<?, ?it/s]

Percent complete: 4.08%


Retrieving ThermoDoc documents:   0%|          | 0/100 [00:00<?, ?it/s]

...

Percent complete: 95.92%


Retrieving ThermoDoc documents:   0%|          | 0/277 [00:00<?, ?it/s]

Percent complete: 97.96%


Retrieving ThermoDoc documents:   0%|          | 0/125 [00:00<?, ?it/s]

Percent complete: 100.00%


In [42]:
matched_entries = []

for mat_id, entries in zip(materials_gga, pbx_entries):
    matched_entry = next((entry for entry in entries if entry.entry_id == mat_id), None)
    
    if matched_entry:
        matched_entries.append(matched_entry)
    else:
        matched_entries.append(None)

In [47]:
material_stabilities = []

for i, x in zip(pbx_data, matched_entries):
    stable_count = 0
    total_pH_count = 0

    for pH in range(3, 13):
        try:
            stability = i.get_decomposition_energy(x, pH, 0)
            total_pH_count += 1
            if stability < 0.5:
                stable_count += 1
        except Exception as e:
            continue
    
    if total_pH_count > 0:
        stability_percentage = (stable_count / total_pH_count) * 100
    else:
        stability_percentage = 0

    material_stabilities.append({'entry': x, 'stability_percentage': stability_percentage})

for material in material_stabilities:
    print(f"Material: {material['entry']}, Stability Percentage: {material['stability_percentage']:.2f}%")


Material: PourbaixEntry(As4 Pb28 Cl8 O34 with energy = -22.8210, npH = -68.0, nPhi = -68.0, nH2O = 34.0, entry_id = 'mp-680722-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(As4 Pb16 O26 with energy = -3.0875, npH = -52.0, nPhi = -52.0, nH2O = 26.0, entry_id = 'mp-31107-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(As2 Au2 O8 with energy = 8.1986, npH = -16.0, nPhi = -16.0, nH2O = 8.0, entry_id = 'mp-1095378-GGA'), Stability Percentage: 80.00%
Material: PourbaixEntry(As2 Se12 I2 F12 with energy = -33.0472, npH = 0, nPhi = 0, nH2O = 0, entry_id = 'mp-23569-GGA'), Stability Percentage: 70.00%
Material: PourbaixEntry(Au8 S8 O32 with energy = 25.7552, npH = -64.0, nPhi = -64.0, nH2O = 32.0, entry_id = 'mp-30003-GGA'), Stability Percentage: 90.00%
Material: PourbaixEntry(Ba1 Au2 S4 O16 with energy = 4.2498, npH = -32.0, nPhi = -32.0, nH2O = 16.0, entry_id = 'mp-1190864-GGA'), Stability Percentage: 100.00%
Material: PourbaixEntry(Bi2 Pb2 Cl2 O4 with energy = -5.0